In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 5)

In [11]:
DATA_PATH = r"E:\Python\Machile Learning\3_Machile Learning\Research\NextEnergy\XAI-ML-SOC-Prediction-TimeSeries\Dataset\Lithium-Ion Battery Drive Cycle Dataset\2_preprocessed"

In [13]:
# Function to load data
def load_all_drive_cycles(base_dir=DATA_PATH):
    data_dict = {}
    for folder in os.listdir(base_dir):
        if not folder.startswith("JY_SOC"):
            continue
        temp = int(folder.replace("JY_SOC_", "").replace("deg", ""))
        temp_path = os.path.join(base_dir, folder)
        data_dict[temp] = {}
        for file in os.listdir(temp_path):
            if file.endswith(".csv"):
                drive_cycle = "_".join(file.replace(".csv", "").split("_")[1:])
                df = pd.read_csv(os.path.join(temp_path, file))
                data_dict[temp][drive_cycle] = df
    return data_dict


In [14]:
drive_data = load_all_drive_cycles()